In [3]:
# libraries

import pandas as pd


In [4]:
# Import Dataset

data = pd.read_csv('./fediverse-client-alt-text-data-2024-05-13.csv')

In [5]:
data

,client,status_count,descriptions_all_count,descriptions_all_percent,descriptions_some_count,descriptions_some_percent,descriptions_none_count,descriptions_none_percent
0,Web,8272,1438,17.383946,7,0.084623,6827,82.531431
1,dlvr.it,5806,1,0.017224,0,0.000000,5805,99.982776
2,Mastodon for Android,1894,270,14.255544,3,0.158395,1621,85.586061
3,unknown,1428,366,25.630252,1,0.070028,1061,74.299720
4,AboveMaidstoneBot,1339,0,0.000000,0,0.000000,1339,100.000000
...,...,...,...,...,...,...,...,...
261,socialbot,1,1,100.000000,0,0.000000,0,0.000000
262,PhonocasterMusicShare,1,0,0.000000,0,0.000000,1,100.000000
263,openvibe,1,0,0.000000,0,0.000000,1,100.000000
264,iflaapp,1,0,0.000000,0,0.000000,1,100.000000


In [6]:
column = data['status_count']

# Check the data type of all values in the column
print(column.dtype)

int64


In [7]:
def summarize_dataframe(df):
    """
    Provides a summary of each column in the DataFrame, including:
    - Data type
    - Number of missing values
    - Number of unique values
    - Descriptive statistics (for numerical columns)
    
    Parameters:
    df (pd.DataFrame): The DataFrame to summarize.
    """
    # Data type and count of missing values
    summary = pd.DataFrame({
        'Data Type': df.dtypes,
        'Missing Values': df.isnull().sum(),
        'Unique Values': df.nunique()
    })
    
    # Descriptive statistics for numerical columns
    numerical_summary = df.describe().T
    summary = summary.merge(numerical_summary, left_index=True, right_index=True, how="left")
    
    # Display summary for categorical columns separately
    categorical_columns = df.select_dtypes(include=['object']).columns
    categorical_summary = df[categorical_columns].describe().T
    
    print("Summary of Numerical Columns:")
    display(summary)
    print("\nSummary of Categorical Columns:")
    display(categorical_summary)


In [8]:
summarize_dataframe(data)

Summary of Numerical Columns:


,Data Type,Missing Values,Unique Values,count,mean,std,min,25%,50%,75%,max
client,object,0,266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
status_count,int64,0,80,266.0,112.240602,643.625189,1.0,2.0,8.0,39.75,8272.000000
descriptions_all_count,int64,0,45,266.0,21.473684,105.282555,0.0,0.0,0.0,4.00,1438.000000
descriptions_all_percent,float64,0,44,266.0,36.290411,45.079671,0.0,0.0,0.0,100.00,100.000000
descriptions_some_count,int64,0,6,266.0,0.093985,0.622382,0.0,0.0,0.0,0.00,7.000000
descriptions_some_percent,float64,0,12,266.0,0.060461,0.527118,0.0,0.0,0.0,0.00,7.142857
descriptions_none_count,int64,0,70,266.0,90.672932,568.477040,0.0,0.0,2.0,20.00,6827.000000
descriptions_none_percent,float64,0,44,266.0,63.649128,45.113843,0.0,0.0,100.0,100.00,100.000000



Summary of Categorical Columns:


,count,unique,top,freq
client,266,266,TZN FS Poster,1
